In [ ]:
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Reshape, Permute, Activation, Input, \
    add, multiply
from keras.layers import concatenate, core, Dropout
from keras.models import Model
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.layers.core import Lambda
import keras.backend as K




def up_and_concate(down_layer, layer, data_format='channels_first'):
    if data_format == 'channels_first':
        in_channel = down_layer.get_shape().as_list()[1]
    else:
        in_channel = down_layer.get_shape().as_list()[3]

    # up = Conv2DTranspose(out_channel, [2, 2], strides=[2, 2])(down_layer)
    up = UpSampling2D(size=(2, 2), data_format=data_format)(down_layer)

    if data_format == 'channels_first':
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=1))
    else:
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=3))

    concate = my_concat([up, layer])

    return concate


def attention_up_and_concate(down_layer, layer, data_format='channels_first'):
    if data_format == 'channels_first':
        in_channel = down_layer.get_shape().as_list()[1]
    else:
        in_channel = down_layer.get_shape().as_list()[3]

    # up = Conv2DTranspose(out_channel, [2, 2], strides=[2, 2])(down_layer)
    up = UpSampling2D(size=(2, 2), data_format=data_format)(down_layer)

    layer = attention_block_2d(x=layer, g=up, inter_channel=in_channel // 4, data_format=data_format)

    if data_format == 'channels_first':
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=1))
    else:
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=3))

    concate = my_concat([up, layer])
    return concate


def attention_block_2d(x, g, inter_channel, data_format='channels_first'):
    # theta_x(?,g_height,g_width,inter_channel)

    theta_x = Conv2D(inter_channel, [1, 1], strides=[1, 1], data_format=data_format)(x)

    # phi_g(?,g_height,g_width,inter_channel)

    phi_g = Conv2D(inter_channel, [1, 1], strides=[1, 1], data_format=data_format)(g)

    # f(?,g_height,g_width,inter_channel)

    f = Activation('relu')(add([theta_x, phi_g]))

    # psi_f(?,g_height,g_width,1)

    psi_f = Conv2D(1, [1, 1], strides=[1, 1], data_format=data_format)(f)

    rate = Activation('sigmoid')(psi_f)

    # rate(?,x_height,x_width)

    # att_x(?,x_height,x_width,x_channel)

    att_x = multiply([x, rate])

    return att_x


def res_block(input_layer, out_n_filters, batch_normalization=False, kernel_size=[3, 3], stride=[1, 1],

              padding='same', data_format='channels_first'):
    if data_format == 'channels_first':
        input_n_filters = input_layer.get_shape().as_list()[1]
    else:
        input_n_filters = input_layer.get_shape().as_list()[3]

    layer = input_layer
    for i in range(2):
        layer = Conv2D(out_n_filters // 4, [1, 1], strides=stride, padding=padding, data_format=data_format)(layer)
        if batch_normalization:
            layer = BatchNormalization()(layer)
        layer = Activation('relu')(layer)
        layer = Conv2D(out_n_filters // 4, kernel_size, strides=stride, padding=padding, data_format=data_format)(layer)
        layer = Conv2D(out_n_filters, [1, 1], strides=stride, padding=padding, data_format=data_format)(layer)

    if out_n_filters != input_n_filters:
        skip_layer = Conv2D(out_n_filters, [1, 1], strides=stride, padding=padding, data_format=data_format)(
            input_layer)
    else:
        skip_layer = input_layer
    out_layer = add([layer, skip_layer])
    return out_layer


# Recurrent Residual Convolutional Neural Network based on U-Net (R2U-Net)
def rec_res_block(input_layer, out_n_filters, batch_normalization=False, kernel_size=[3, 3], stride=[1, 1],

                  padding='same', data_format='channels_first'):
    if data_format == 'channels_first':
        input_n_filters = input_layer.get_shape().as_list()[1]
    else:
        input_n_filters = input_layer.get_shape().as_list()[3]

    if out_n_filters != input_n_filters:
        skip_layer = Conv2D(out_n_filters, [1, 1], strides=stride, padding=padding, data_format=data_format)(
            input_layer)
    else:
        skip_layer = input_layer

    layer = skip_layer
    for j in range(2):

        for i in range(2):
            if i == 0:

                layer1 = Conv2D(out_n_filters, kernel_size, strides=stride, padding=padding, data_format=data_format)(
                    layer)
                if batch_normalization:
                    layer1 = BatchNormalization()(layer1)
                layer1 = Activation('relu')(layer1)
            layer1 = Conv2D(out_n_filters, kernel_size, strides=stride, padding=padding, data_format=data_format)(
                add([layer1, layer]))
            if batch_normalization:
                layer1 = BatchNormalization()(layer1)
            layer1 = Activation('relu')(layer1)
        layer = layer1

    out_layer = add([layer, skip_layer])
    return out_layer

########################################################################################################
# Define the neural network
def unet(img_w, img_h, n_label, data_format='channels_first'):
    inputs = Input((3, img_w, img_h))
    x = inputs
    depth = 4
    features = 64
    skips = []
    for i in range(depth):
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        x = Dropout(0.2)(x)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        skips.append(x)
        x = MaxPooling2D((2, 2), data_format= data_format)(x)
        features = features * 2

    x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
    x = Dropout(0.2)(x)
    x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)

    for i in reversed(range(depth)):
        features = features // 2
        # attention_up_and_concate(x,[skips[i])
        x = UpSampling2D(size=(2, 2), data_format=data_format)(x)
        x = concatenate([skips[i], x], axis=1)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        x = Dropout(0.2)(x)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)

    conv6 = Conv2D(n_label, (1, 1), padding='same', data_format=data_format)(x)
    conv7 = core.Activation('sigmoid')(conv6)
    model = Model(inputs=inputs, outputs=conv7)

    #model.compile(optimizer=Adam(lr=1e-5), loss=[focal_loss()], metrics=['accuracy', dice_coef])
    return model


########################################################################################################
#Attention U-Net
def att_unet(img_w, img_h, n_label, data_format='channels_first'):
    inputs = Input((3, img_w, img_h))
    x = inputs
    depth = 4
    features = 64
    skips = []
    for i in range(depth):
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        x = Dropout(0.2)(x)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        skips.append(x)
        x = MaxPooling2D((2, 2), data_format='channels_first')(x)
        features = features * 2

    x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
    x = Dropout(0.2)(x)
    x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)

    for i in reversed(range(depth)):
        features = features // 2
        x = attention_up_and_concate(x, skips[i], data_format=data_format)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        x = Dropout(0.2)(x)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)

    conv6 = Conv2D(n_label, (1, 1), padding='same', data_format=data_format)(x)
    conv7 = core.Activation('sigmoid')(conv6)
    model = Model(inputs=inputs, outputs=conv7)

    #model.compile(optimizer=Adam(lr=1e-5), loss=[focal_loss()], metrics=['accuracy', dice_coef])
    return model


########################################################################################################
#Recurrent Residual Convolutional Neural Network based on U-Net (R2U-Net)
def r2_unet(img_w, img_h, n_label, data_format='channels_first'):
    inputs = Input((3, img_w, img_h))
    x = inputs
    depth = 4
    features = 64
    skips = []
    for i in range(depth):
        x = rec_res_block(x, features, data_format=data_format)
        skips.append(x)
        x = MaxPooling2D((2, 2), data_format=data_format)(x)

        features = features * 2

    x = rec_res_block(x, features, data_format=data_format)

    for i in reversed(range(depth)):
        features = features // 2
        x = up_and_concate(x, skips[i], data_format=data_format)
        x = rec_res_block(x, features, data_format=data_format)

    conv6 = Conv2D(n_label, (1, 1), padding='same', data_format=data_format)(x)
    conv7 = core.Activation('sigmoid')(conv6)
    model = Model(inputs=inputs, outputs=conv7)
    #model.compile(optimizer=Adam(lr=1e-6), loss=[dice_coef_loss], metrics=['accuracy', dice_coef])
    return model


########################################################################################################
#Attention R2U-Net
def att_r2_unet(img_w, img_h, n_label, data_format='channels_last'):
    inputs = Input((img_w, img_h , 3))
    x = inputs
    depth = 4
    features = 64
    skips = []
    for i in range(depth):
        x = rec_res_block(x, features, data_format=data_format)
        skips.append(x)
        x = MaxPooling2D((2, 2), data_format=data_format)(x)

        features = features * 2

    x = rec_res_block(x, features, data_format=data_format)

    for i in reversed(range(depth)):
        features = features // 2
        x = attention_up_and_concate(x, skips[i], data_format=data_format)
        x = rec_res_block(x, features, data_format=data_format)

    conv6 = Conv2D(n_label, (1, 1), padding='same', data_format=data_format)(x)
    conv7 = core.Activation('sigmoid')(conv6)
    model = Model(inputs=inputs, outputs=conv7)
    #model.compile(optimizer=Adam(lr=1e-6), loss=[dice_coef_loss], metrics=['accuracy', dice_coef])
    return model

In [ ]:
import os
import numpy as np
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cv2

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from skimage import io # To read a single image

datagen = ImageDataGenerator( #Randomly rotates but with atmost rotation as 45 degree , Randomly width is varied but with atmost 20% width change 
                        rotation_range = 45 , 
                        width_shift_range = 0.2,
                        height_shift_range = 0.2 ,
                        shear_range        = 0.2 , 
                        horizontal_flip = True , 
                        fill_mode = "constant" , cval = 125)
)
def load_data(paths , split = 0.15):
  images = []
  masks = []
  numImages = 0
  numMasks = 0

  for path in paths:
    
        
    images.extend(sorted(glob(os.path.join(path , "images/*"))))
    masks.extend(sorted(glob(os.path.join(path , "label/*"))))
    if "training" in path:
      for i in ["00", "01" , "02" , "03" , "04" , "05" , "07", "08" , "09" , "10" , "11" , "12" , "13" , "14" , "16" , "17" , "18" , "19" , "21" , "22" , "24" , "26" , "27" ]:
        images.remove("../input/chagas/ChagasTraining/training-20210525T143718Z-001/training/images/i8{}.xml".format(i))

      n = "training"
    if "Test" in path:
      n= "Test"
    if "Val" in path:

      n= "Validation"

    print("{}PathImagesAre:".format(n),len(images) - numImages)
    print("{}PathImagesAre:".format(n),len(masks) - numMasks)

    numImages = len(images)
    numMasks = len(masks)

  total_size = len(images)
  valid_size = int(split * total_size)
  test_size = int(split * total_size)
  print("Number of images in Total , Validation images and Test Images : " , total_size , valid_size ,test_size  )

  train_x , test_x = train_test_split(images , test_size = test_size , random_state = 42 )
  train_y , test_y = train_test_split(masks , test_size = test_size , random_state = 42 )

  train_x , valid_x = train_test_split(train_x , test_size = valid_size , random_state = 42 )
  train_y , valid_y = train_test_split(train_y , test_size = valid_size , random_state = 42 )

  return (train_x , train_y) , (valid_x , valid_y ) , (test_x , test_y) 

def read_image(path):
  path = path.decode()
  x = cv2.imread(path , cv2.IMREAD_COLOR)
  #x = cv2.resize(x , (256 , 256))
  x = x/255.0
  #Size is 256 * 256 * 3

  return x

def read_mask(path):
  path = path.decode()
  x = cv2.imread(path , cv2.IMREAD_GRAYSCALE)
  #x = cv2.resize(x , (256 , 256))
  x = x/255.0
  
  #Size is 256 * 256
  x = np.expand_dims(x , axis = -1)
  #Size becames 256 * 256 * 1

  return x

def tf_parse(imagepath , maskpath):
  def _parse(imagepath , maskpath):
    x = read_image(imagepath)
    y = read_mask(maskpath)

    return x , y

  x , y = tf.numpy_function(_parse , [imagepath , maskpath] , [tf.float64 , tf.float64] )
  x.set_shape([512 , 512 , 3])
  y.set_shape([512, 512, 1])

  return x , y 


def tf_dataset( imagepath , maskpath , batch = 2):
  dataset = tf.data.Dataset.from_tensor_slices((imagepath , maskpath))
  dataset = dataset.map(tf_parse)
  dataset = dataset.batch(batch)
  dataset = dataset.repeat()
  return dataset

if __name__ == "__main__":
  paths = ["../input/chagas/ChagasTraining/training-20210525T143718Z-001/training" , "../input/chagas/ChagasTest/Test", "../input/chagas/ChagasValidation/Validation"] 
  (train_x , train_y) , (valid_x , valid_y ) , (test_x , test_y)  =   load_data(paths)

  print("Number of images in Train , Validation images and Test Images : " ,len(train_x) , len(valid_x ) , len(test_x) )

  ds = tf_dataset(test_x , test_y)
  for x, y in ds:
    
    print(x.shape , y.shape)
    break
    

trainingPathImagesAre: 600
trainingPathImagesAre: 600
TestPathImagesAre: 200
TestPathImagesAre: 200
ValidationPathImagesAre: 200
ValidationPathImagesAre: 200
Number of images in Total , Validation images and Test Images :  1000 150 150
Number of images in Train , Validation images and Test Images :  700 150 150
(2, 512, 512, 3) (2, 512, 512, 1)


In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from tensorflow.keras.metrics import Recall , Precision 

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

In [ ]:
pip install segmentation_models


Note: you may need to restart the kernel to use updated packages.


In [ ]:
%env SM_FRAMEWORK=tf.keras


env: SM_FRAMEWORK=tf.keras


In [ ]:
from keras import backend as K
from keras.losses import binary_crossentropy
import tensorflow as tf
import numpy as np
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score
def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss
def iou_coeff(y_true, y_pred):
    smooth=1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union=K.sum(y_true_f) + K.sum(y_pred_f)-intersection
    mvalue=(intersection+smooth)/(union+smooth)
    return mvalue
def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
def ACL5(y_true, y_pred): 

    #y_pred = K.cast(y_pred, dtype = 'float64')

    print(K.int_shape(y_pred))

    x = y_pred[:,1:,:,:] - y_pred[:,:-1,:,:] # horizontal and vertical directions 
    y = y_pred[:,:,1:,:] - y_pred[:,:,:-1,:]

    delta_x = x[:,1:,:-2,:]**2
    delta_y = y[:,:-2,1:,:]**2
    delta_u = K.abs(delta_x + delta_y) 

    epsilon = 0.00000001 # where is a parameter to avoid square root is zero in practice.
    w = 1####
    lenth = w * K.sum(K.sqrt(delta_u + epsilon)) # equ.(11) in the paper


    C_1 = tf.ones((512, 512))
    C_2 = tf.zeros((512, 512))

    region_in = K.abs(K.sum( y_pred[:,:,:,0] * ((y_true[:,:,:,0] - C_1)**2) ) ) # equ.(12) in the paper
    region_out = K.abs(K.sum( (1-y_pred[:,:,:,0]) * ((y_true[:,:,:,0] - C_2)**2) )) # equ.(12) in the paper

    lambdaP = 5 # lambda parameter could be various.
    
    loss =  lenth + lambdaP * ((region_in) + (region_out)) 

    return loss

In [ ]:
if __name__ == "__main__":
    img_w, img_h, n_label = 512 , 512 , 1
    model = att_r2_unet(img_w, img_h, n_label, data_format='channels_last')
    
    model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 64) 256         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 512, 512, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [ ]:
import segmentation_models
if __name__ == "__main__":
    from segmentation_models.losses import bce_jaccard_loss
    from segmentation_models.metrics import iou_score
    np.random.seed(42)
    tf.random.set_seed(42)
    #HYPERPARAMETER


    batch = 2
    lr = 8.00E-05
    epochs = 2
    opt = tf.keras.optimizers.Adam(lr)
    metrics = ["acc" , Recall() , Precision() , iou]
    #Data
    train_dataset = tf_dataset(train_x, train_y, batch=batch)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)
    img_w, img_h, n_label = 512 , 512 , 1
    model = att_r2_unet(img_w, img_h, n_label, data_format='channels_last')
    #final_output , model = build_model((512 , 512 , 3))
    #model.summary()
    #model.compile(loss=bce_jaccard_loss, optimizer=opt, metrics=metrics)
    model.compile(optimizer= opt, loss=bce_jaccard_loss, metrics=[dice_loss,iou_coeff,precision,recall])
    
    callbacks = [ 
        ModelCheckpoint("files/model.h5"),
        ReduceLROnPlateau(monitor='iou', factor=0.1, patience=4),
        #CSVLogger("/content/drive/MyDrive/data.csv"),
        TensorBoard(),
        EarlyStopping(monitor='iou', patience=15, restore_best_weights=False)
    ]
    #Number Of Batches
    train_steps = len(train_x)//batch
    valid_steps = len(valid_x)//batch

    if len(train_x) % batch != 0:
        train_steps += 1
    if len(valid_x) % batch != 0:
        valid_steps += 1

    model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=epochs,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks,
        shuffle = False)

Segmentation Models: using `tf.keras` framework.
Epoch 1/2
350/350 [==============================] - 256s 711ms/step - loss: 1.0154 - dice_loss: 0.8507 - iou_coeff: 0.0951 - precision: 0.2142 - recall: 0.1673 - val_loss: 0.5629 - val_dice_loss: 0.3617 - val_iou_coeff: 0.4815 - val_precision: 0.7249 - val_recall: 0.6418
Epoch 2/2
350/350 [==============================] - 248s 709ms/step - loss: 0.6153 - dice_loss: 0.4088 - iou_coeff: 0.4403 - precision: 0.6848 - recall: 0.6196 - val_loss: 0.6490 - val_dice_loss: 0.4513 - val_iou_coeff: 0.4038 - val_precision: 0.8161 - val_recall: 0.4445


In [ ]:
    model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=30,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks,
        shuffle = False)

Epoch 1/30
350/350 [==============================] - 249s 710ms/step - loss: 0.5363 - dice_loss: 0.3371 - iou_coeff: 0.5110 - precision: 0.7182 - recall: 0.6970 - val_loss: 0.5006 - val_dice_loss: 0.3081 - val_iou_coeff: 0.5413 - val_precision: 0.7985 - val_recall: 0.6474
Epoch 2/30
350/350 [==============================] - 248s 709ms/step - loss: 0.5143 - dice_loss: 0.3243 - iou_coeff: 0.5293 - precision: 0.7277 - recall: 0.7082 - val_loss: 0.5027 - val_dice_loss: 0.3123 - val_iou_coeff: 0.5445 - val_precision: 0.8357 - val_recall: 0.6224
Epoch 3/30
350/350 [==============================] - 248s 709ms/step - loss: 0.5014 - dice_loss: 0.3111 - iou_coeff: 0.5423 - precision: 0.7289 - recall: 0.7306 - val_loss: 0.4274 - val_dice_loss: 0.2548 - val_iou_coeff: 0.6075 - val_precision: 0.7591 - val_recall: 0.7803
Epoch 4/30
350/350 [==============================] - 248s 709ms/step - loss: 0.4707 - dice_loss: 0.2920 - iou_coeff: 0.5698 - precision: 0.7612 - recall: 0.7273 - val_loss: 0.46

In [ ]:

    model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=20,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks,
        shuffle = False)

Epoch 1/20
350/350 [==============================] - 248s 710ms/step - loss: 0.2308 - dice_loss: 0.1223 - iou_coeff: 0.7867 - precision: 0.8940 - recall: 0.8832 - val_loss: 0.3842 - val_dice_loss: 0.2253 - val_iou_coeff: 0.6517 - val_precision: 0.8241 - val_recall: 0.7731
Epoch 2/20
350/350 [==============================] - 248s 709ms/step - loss: 0.2138 - dice_loss: 0.1118 - iou_coeff: 0.8030 - precision: 0.9028 - recall: 0.8934 - val_loss: 0.3845 - val_dice_loss: 0.2212 - val_iou_coeff: 0.6536 - val_precision: 0.8127 - val_recall: 0.7896
Epoch 3/20
350/350 [==============================] - 248s 709ms/step - loss: 0.2020 - dice_loss: 0.1052 - iou_coeff: 0.8137 - precision: 0.9099 - recall: 0.8995 - val_loss: 0.3800 - val_dice_loss: 0.2187 - val_iou_coeff: 0.6565 - val_precision: 0.8435 - val_recall: 0.7674
Epoch 4/20
350/350 [==============================] - 248s 709ms/step - loss: 0.2077 - dice_loss: 0.1082 - iou_coeff: 0.8091 - precision: 0.9056 - recall: 0.8975 - val_loss: 0.38